In [4]:
import numpy as np
from os.path import join
data_path = join('./','data')
# emb
with open(join(data_path,'t1.emb')) as f:
    num_nodes, D = f.readline().strip().split(' ')
    num_nodes = int(num_nodes)
    D = int(D)
    
    ls = f.readlines()
node_emb_dict = {}
for l in ls:
    buf = l.strip().split(' ')
    node_id, emb = int(buf[0]), buf[1:]
    x = np.asarray([float(i) for i in emb], dtype=np.float32)
    node_emb_dict[node_id] = x
    
# training data
with open(join(data_path,'t1-merge.txt')) as f:
    ls = f.readlines()
N = len(ls)
X = []
adj_mat = np.zeros([N,N], dtype=np.uint8)
for l in ls:
    buf = l.strip().split(' ')
    src, dst = int(buf[0]), int(buf[1])
    adj_mat[src, dst] = 1
    fea = np.concatenate([node_emb_dict[src], node_emb_dict[dst]], axis=-1)
    X.append(fea)
X = np.vstack(X)

# test data
with open(join(data_path,'t1-test.txt')) as f:
    ls = f.readlines()
N2 = len(ls)
test_X = []
for l in ls:
    buf = l.strip().split(' ')
    src, dst = int(buf[0]), int(buf[1])
    if src not in node_emb_dict:
        src = 27026
    if dst not in node_emb_dict:
        dst = 101
    fea = np.concatenate([node_emb_dict[src], node_emb_dict[dst]], axis=-1)
    
    test_X.append(fea)
test_X = np.vstack(test_X)
print X.shape, test_X.shape
print 'done'
    

(285789, 256) (88074, 256)
done


In [5]:
import numpy as np
batch_size = 128
def naive_bootsrap_generator(X, adj_mat, node_emb_dict, batch_size=128, neg_rate=1.):
    exist_node_list = node_emb_dict.keys()
    exist_N = len(exist_node_list)
    num_edge = X.shape[0]
        
    while True:
        idx = np.random.choice(num_edge, batch_size)
        pos_X = X[idx, :]
        
        neg_count = int(batch_size*neg_rate)
        neg_idx = np.random.randint(exist_N, size=[neg_count, 2])
        neg_X = []
        for i in range(neg_count):
            src, dst = neg_idx[i]
            src = exist_node_list[src]
            dst = exist_node_list[dst]
            if src != dst and adj_mat[src, dst] == 0:
                fea = np.concatenate([node_emb_dict[src], node_emb_dict[dst]], axis=-1)
                neg_X.append(fea)
        neg_X = np.vstack(neg_X)

        ret_X = np.vstack([pos_X, neg_X])
        ret_Y = np.zeros([ret_X.shape[0], 1])
        ret_Y[:batch_size, 0] = 1
        yield ret_X, ret_Y
N = X.shape[0]
idx = np.random.permutation(N)
train_idx = idx[N//10:]
val_idx = idx[:N//10]

train_X = X[train_idx,:]
val_X = X[val_idx,:]
G = naive_bootsrap_generator(train_X, adj_mat, node_emb_dict, batch_size=batch_size)
val_G = naive_bootsrap_generator(val_X, adj_mat, node_emb_dict, batch_size=batch_size, neg_rate=0.1)
x,y = next(G)
print x.shape,y.shape
x,y = next(val_G)
print x.shape,y.shape

(256, 256) (256, 1)
(140, 256) (140, 1)


In [3]:
import keras
from keras.models import *
from keras.layers import *

epochs = 100
def build_model():

    model = Sequential()
    model.add(Dense(256, activation='selu', input_shape=(256,)))
    model.add(Dense(256, activation='selu'))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='selu'))
    model.add(Dense(128, activation='selu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='selu'))
    model.add(Dense(64, activation='selu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='selu'))
    model.add(Dense(32, activation='selu'))
    model.add(Dropout(0.2))
    model.add(Dense(16, activation='selu'))
    model.add(Dense(16, activation='selu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])
    return model


Using TensorFlow backend.


In [ ]:
x,y = next(G)
print x.shape, y.shape
np.random.seed(1337)
model = build_model()
model.fit(x,y)
ck = keras.callbacks.ModelCheckpoint('./weights.hdf5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
tfb = keras.callbacks.TensorBoard(log_dir='./logs')
model.fit_generator(G,
                    steps_per_epoch=train_X.shape[0]//batch_size,
                    epochs=1000, verbose=1,
                    validation_data=val_G,
                    validation_steps=val_X.shape[0]//batch_size,
                    callbacks=[ck,tfb]
                    )


(256, 256) (256, 1)
Epoch 1/1
256/256 [==============================] - 1s 4ms/step - loss: 0.7893 - acc: 0.5391
Epoch 1/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.2238 - acc: 0.9067 - val_loss: 0.0795 - val_acc: 0.9720
Epoch 2/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.1149 - acc: 0.9599 - val_loss: 0.1037 - val_acc: 0.9757
Epoch 3/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.0528 - acc: 0.9829 - val_loss: 0.0755 - val_acc: 0.9843
Epoch 11/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.0512 - acc: 0.9835 - val_loss: 0.0457 - val_acc: 0.9886
Epoch 12/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.0477 - acc: 0.9847 - val_loss: 0.0542 - val_acc: 0.9856
Epoch 13/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.0469 - acc: 0.9852 - val_loss: 0.0689 - val_acc: 0.9773
Epoch 14/1000
2009/2009 [==============================] - 9s 5ms/s

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2009/2009 [==============================] - 9s 5ms/step - loss: 0.0266 - acc: 0.9924 - val_loss: 0.0372 - val_acc: 0.9873
Epoch 59/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.0260 - acc: 0.9928 - val_loss: 0.0374 - val_acc: 0.9870
Epoch 60/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.0246 - acc: 0.9931 - val_loss: 0.0355 - val_acc: 0.9894
Epoch 61/1000
2009/2009 [==============================] - 10s 5ms/step - loss: 0.0240 - acc: 0.9933 - val_loss: 0.0707 - val_acc: 0.9859
Epoch 62/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.0242 - acc: 0.9934 - val_loss: 0.0359 - val_acc: 0.9909
Epoch 63/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.0237 - acc: 0.9934 - val_loss: 0.0506 - val_acc: 0.9835
Epoch 64/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.0236 - acc: 0.9935 - val_loss: 0.0587 - val_acc: 0.9797
Epoch 65/1000
2009/2009 [=============================

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2009/2009 [==============================] - 9s 5ms/step - loss: 0.0245 - acc: 0.9930 - val_loss: 0.0278 - val_acc: 0.9930
Epoch 106/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.0240 - acc: 0.9933 - val_loss: 0.0444 - val_acc: 0.9886
Epoch 107/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.0246 - acc: 0.9931 - val_loss: 0.0363 - val_acc: 0.9912
Epoch 108/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.0222 - acc: 0.9939 - val_loss: 0.0408 - val_acc: 0.9875
Epoch 109/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.0259 - acc: 0.9928 - val_loss: 0.0456 - val_acc: 0.9874
Epoch 110/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.0264 - acc: 0.9928 - val_loss: 0.0332 - val_acc: 0.9917
Epoch 111/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.0239 - acc: 0.9934 - val_loss: 0.1174 - val_acc: 0.9489
Epoch 112/1000
2009/2009 [=======================

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2009/2009 [==============================] - 9s 5ms/step - loss: 0.0191 - acc: 0.9952 - val_loss: 0.1031 - val_acc: 0.9716
Epoch 154/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.0200 - acc: 0.9950 - val_loss: 0.0998 - val_acc: 0.9801
Epoch 155/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.0320 - acc: 0.9931 - val_loss: 0.1219 - val_acc: 0.9838
Epoch 156/1000
2009/2009 [==============================] - 10s 5ms/step - loss: 0.0326 - acc: 0.9918 - val_loss: 0.0701 - val_acc: 0.9785
Epoch 157/1000
2009/2009 [==============================] - 10s 5ms/step - loss: 0.0208 - acc: 0.9944 - val_loss: 0.0516 - val_acc: 0.9866
Epoch 158/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.0209 - acc: 0.9946 - val_loss: 0.0547 - val_acc: 0.9809
Epoch 159/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.0306 - acc: 0.9913 - val_loss: 0.0686 - val_acc: 0.9758
Epoch 160/1000
2009/2009 [=====================

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2009/2009 [==============================] - 9s 5ms/step - loss: 0.0189 - acc: 0.9955 - val_loss: 0.1296 - val_acc: 0.9790
Epoch 200/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.0340 - acc: 0.9946 - val_loss: 0.1731 - val_acc: 0.9749
Epoch 201/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.0330 - acc: 0.9921 - val_loss: 0.0914 - val_acc: 0.9790
Epoch 202/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.0218 - acc: 0.9948 - val_loss: 0.1192 - val_acc: 0.9801
Epoch 203/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.0211 - acc: 0.9951 - val_loss: 0.1090 - val_acc: 0.9848
Epoch 204/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.0848 - acc: 0.9870 - val_loss: 0.1016 - val_acc: 0.9748
Epoch 205/1000
2009/2009 [==============================] - 9s 5ms/step - loss: 0.0300 - acc: 0.9927 - val_loss: 0.0806 - val_acc: 0.9799
Epoch 206/1000
2009/2009 [=======================

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
from keras.models import *
model = load_model('./weights.hdf5')
z = model.predict(test_X)
with open('pred.txt', 'w') as f:
    for i in range(z.shape[0]):
        p = z[i,0]
        ans = 1 if p >= 0.5 else 0
        f.write('%d\n' % ans)
pred_file = 'pred.txt'
with open(pred_file, 'r') as f, open(pred_file + '.csv', 'w') as g:
    g.write('query_id,prediction\n')
    for idx, line in enumerate(f):
        g.write('%d,%d\n' % (1 + idx, int(line)))


print 'done'
print z.shape

done
(88074, 1)


In [ ]:
import tensorflow
import keras
print tensorflow.__version__
print keras.__version__

In [ ]:
88074-38128
97232 - 88074